# Moteur de recherche

In [1]:
from transformers import CamembertTokenizer, CamembertModel
import torch
import numpy as np
import pandas as pd
from scipy.spatial.distance import cosine

In [2]:
# Chargement du modèle et du tokenizer CamemBERT
model_name = 'camembert-base'
tokenizer = CamembertTokenizer.from_pretrained(model_name)
model = CamembertModel.from_pretrained(model_name)

c:\Users\julie\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\julie\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at camembert-base were not used when initializing CamembertModel: ['lm_head.layer_

In [3]:
# Fonction pour obtenir l'encodage Camembert du corpus
def encode_corpus(corpus):
    corpus_embeddings = []
    for text in corpus:
        input_ids = tokenizer.encode(text, add_special_tokens=True, return_tensors='pt')
        with torch.no_grad():
            outputs = model(input_ids)
            embeddings = outputs[0][:, 0, :].squeeze().numpy()
            corpus_embeddings.append(embeddings)

    return corpus_embeddings

# Fonction pour calculer la similarité cosinus entre deux vecteurs
def cosine_similarity(vector1, vector2):
    return 1 - cosine(vector1, vector2)

In [4]:
def search_similar_texts(input_text, corpus_embeddings, corpus, n=5):
    # Encodage du texte d'entrée
    input_ids = tokenizer.encode(input_text, add_special_tokens=True, return_tensors='pt')
    with torch.no_grad():
        outputs = model(input_ids)
        input_embedding = outputs[0][:, 0, :].squeeze().numpy()

    # Calcul de la similarité cosinus entre le texte d'entrée et chaque texte du corpus
    similarities = [cosine_similarity(input_embedding, embedding) for embedding in corpus_embeddings]

    # Trie les indices des textes du corpus par ordre de similarité décroissante
    sorted_indices = np.argsort(similarities)[::-1]

    # Récupère les n textes les plus similaires
    similar_texts = [(corpus[i], similarities[i]) for i in sorted_indices[:n]]

    return similar_texts


In [5]:
# Encodage du corpus
data = pd.read_csv("data/data.csv", index_col=0)
Titres = data.titre.values

Titres_embedings=encode_corpus(corpus=Titres)

In [11]:
# Texte d'entrée en français
input_text = "Sociologie du travail rural"

# Recherche des 3 textes les plus similaires, possible de selectionner le nombre de texte similaires manuellement
similar_texts = search_similar_texts(input_text, corpus_embeddings=Titres_embedings,corpus=Titres, n=3)

# Affichage des textes les plus similaires
print("Textes les plus similaires :")
for text, similarity in similar_texts:
    print(text, "Similarity:", similarity)

Textes les plus similaires :
Gilles Ferréol, Philippe Deubel, Économie du travail Similarity: 0.9780381917953491
François Perroux, Économie et société Similarity: 0.9704778790473938
Reconversion des élites agraires.  Similarity: 0.9692001342773438
